## Todo List
- [x] Checar se formato de datas conferem.
- [x] Remover UGs sem liberação OC
- [x] Considerar coluna montagem. Onde estiver vazio, pegar valor de Torres para EOL, no skate.
- [ ] Separar um cojunto aleatório para teste
- [ ] Calcular tempos de construção por marco: (IO_realizado - OC_realizado), (CC_realizado - OC_realizado) ...etc  (ordem:IO,CC,ME,OT)
- [ ] Calcular atraso da operação comercial (OC_realizado - OC_outorgado)
- [ ] Calcular média e dp do tempo de construção.
- [ ] Calcular data dos marcos medios (OC_outorgado - tempo_medio_de_construcao) por fonte.
- [ ] Calcular atrasos com base nos marcos medios
- [ ] Calcular media e dp dos atrasos. 
- [ ] Normalizar os atrasos calculados (atraso - atraso_medio)/dp
- [ ] Construir matriz de correlação de Pearson
- [ ] Calcular os coeficientes para regressão linear
- [ ] Para UHE pegar todas UGs, para as demais pegar a primeira UG apenas 
- [ ] Comparar pegando primeira UG e media por grupo



## Importa bibliotecas

In [1]:
import pandas as pd
import numpy as np
import re
import pyodbc
from pprint import pprint
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from scipy import stats

##  Cria conexões com os bancos de dado

## Baixando banco de dados

### Corrigindo banco de dados bdworgs


In [2]:
def corrige_ug(ug): # Função criada para corrigir nomes das UGs
    ug = ug.replace(" ","")
    ug = ug.replace("e",",")
    ug = ug.replace("-","a")
    ug = re.sub(r",+",r",",ug)
    ug = re.sub(r"([,ae])(0)",r"\1",ug)
    ug = re.sub(r"^0",r"",ug)
    ug = ug.split("a")
    lista = []
    for i in range(len(ug)-1):
        menor = int(ug[i].split(",")[-1])
        maior = int(ug[i+1].split(",")[0])
        list_nums = list(range(menor+1,maior))
        lista += list_nums
    ug = [el.split(",") for el in ug]
    ug = [int(el2) for el1 in ug for el2 in el1]
    ug+= lista
    return (sorted(ug))

### Carrega bancos de dados

In [3]:
# Carrega bdworgs
bdworgs = pd.read_feather("./bdworgs_corrigido.feather")

# Carrega dois bancos de dados do skate
skate_ug = pd.read_feather("./skate_ug.feather")
skate_usinas = pd.read_feather("./skate_usinas.feather")

### Tabela com associações de nomes entre os bancos de dados

In [4]:
# Carrega tabela de associações
associassoes = pd.read_excel("associassoes.xlsx")
associassoes

,bdworgs,skate
0,Idempreendimento,IdeUsinaOutorga
1,OpTest_Outor_UG,DatInicioOpTesteOutorgada
2,OpCom_Outor_UG,DatUGInicioOpComerOutorgado
3,OpTest_R_UG,DatLiberacaoSFGTeste
4,OpCom_R_UG,DatLiberOpComerRealizado
5,NaN,DatOutorgaUsina
6,UF,UFUsina
7,NaN,SigTipoGeracao
8,PotenciaOutorgada,VlrPotenciaOutorgada
9,Obras_Outorga,DatInicioObraOutorgado


In [5]:
# Cria dicionário dos nomes das colunas para poder relacionar os nomes das colunas
# Dicionário de colunas bdworgs > skate
dict_bdworgs_skate = dict(zip(associassoes.dropna()["bdworgs"],associassoes.dropna()["skate"]))
# Dicionário de colunas skate > bdworgs
dict_skate_bdworgs = dict(zip(associassoes.dropna()["skate"],associassoes.dropna()["bdworgs"]))

In [6]:
# Colunas carregados do skate
skate_cols = [el for el in associassoes["skate"].dropna()]
# Colunas carregadas do bdworgs
bdworgs_cols =[el for el in associassoes["bdworgs"].dropna()]
skate_ug_cols = []
skate_usinas_col = []
for col in skate_cols:
    if col in skate_ug.columns:
        skate_ug_cols.append(col)
    if col in skate_usinas.columns:
        skate_usinas_col.append(col)
        
bdworgs = bdworgs[bdworgs_cols]
skate_ug = skate_ug[skate_ug_cols]
skate_usinas = skate_usinas[skate_usinas_col]
bdworgs = bdworgs.rename(columns=dict_bdworgs_skate)
bdworgs.sample(5)

,IdeUsinaOutorga,DatInicioOpTesteOutorgada,DatUGInicioOpComerOutorgado,DatLiberacaoSFGTeste,DatLiberOpComerRealizado,UFUsina,VlrPotenciaOutorgada,DatInicioObraOutorgado,DatInicioObraRealizado,DatConcretagemOutorgado,...,DatMontagemOutorgado,DatMontagemRealizado,DatConclusaoTorresOutorgado,DatConclusaoTorresRealizado,DatEnchimentoOutorgado,DatEnchimentoRealizado,DatSisTransmissaoOutorgado,DatSisTransmissaoRealizado,Num_UG,Pot_UG
404,30976,None,2014-03-30,2014-09-10,2014-10-03,BA,30.0000,None,2012-02-13,None,...,None,2013-03-28,None,2012-08-13,None,None,None,2012-04-27,1 a 15,30.0000
2086,29711,2009-10-15,2009-11-30,2011-08-17,2011-11-22,SC,3.9006,2008-06-10,2008-04-10,2008-07-28,...,2008-11-24,2011-01-26,None,None,2009-10-10,2011-06-26,None,None,1 e 2,1.9465
1907,29260,2018-09-15,2018-10-15,2018-12-11,2019-01-18,MS,19.0000,2016-07-04,2016-11-01,2017-02-14,...,2017-06-21,2018-06-15,None,None,2018-03-07,None,2017-03-17,2017-05-21,01,9.5000
1789,28996,2006-11-01,2006-12-01,2009-08-14,2009-09-18,MT,24.0000,2005-04-01,2008-01-07,2005-11-01,...,2006-01-01,2008-06-05,None,None,None,2009-02-01,None,None,2 e 3,16.0000
1130,33680,2017-11-18,2017-12-31,2017-05-10,2017-08-03,MA,27.6000,2017-03-31,2016-06-30,2017-06-16,...,None,None,2017-07-06,2017-02-28,None,None,2017-04-16,2016-06-30,7,2.3000


### Adiciona as seguintes  colunas no bdworgs:
- Quant_UG  &rarr; Quantidade de UGs agrupadas
- SigTipoGeracao &rarr; Tipo de geração


In [7]:
# Insere coluna Quant_UG em bdworgs
bdworgs.insert(3,"Quant_UG",bdworgs["Num_UG"].apply(lambda x : len(corrige_ug(x))))
# Remove coluna Num_UG
bdworgs = bdworgs.drop("Num_UG",axis=1)

ceg_df = pd.read_excel("./Relatorio_Outorgas.xlsx")[["Identificador da Usina","Tipo de Geração"]]
dict_id_ceg = dict(zip(ceg_df["Identificador da Usina"],ceg_df["Tipo de Geração"]))
bdworgs["SigTipoGeracao"] = bdworgs["IdeUsinaOutorga"].map(dict_id_ceg)
bdworgs.sample(5)

,IdeUsinaOutorga,DatInicioOpTesteOutorgada,DatUGInicioOpComerOutorgado,Quant_UG,DatLiberacaoSFGTeste,DatLiberOpComerRealizado,UFUsina,VlrPotenciaOutorgada,DatInicioObraOutorgado,DatInicioObraRealizado,...,DatMontagemOutorgado,DatMontagemRealizado,DatConclusaoTorresOutorgado,DatConclusaoTorresRealizado,DatEnchimentoOutorgado,DatEnchimentoRealizado,DatSisTransmissaoOutorgado,DatSisTransmissaoRealizado,Pot_UG,SigTipoGeracao
2192,30004,2009-03-30,2009-04-15,1,2009-12-29,2010-01-16,SP,30.0,None,2009-07-15,...,2009-02-05,None,None,None,None,None,None,None,30.0,UTE
460,31241,2015-07-11,2015-10-21,1,2016-06-23,2016-10-14,CE,218.0,None,2014-01-01,...,None,2014-12-01,None,None,None,None,None,2014-04-01,100.0,UTE
1969,29546,None,None,1,2016-04-05,2016-05-14,GO,24.0,None,None,...,None,None,None,None,None,None,None,None,12.0,UTE
76,30456,2012-09-01,None,1,2013-02-07,2013-03-01,BA,30.0,2011-11-01,2011-11-10,...,None,2012-09-05,2012-03-01,2012-08-25,None,None,None,None,30.0,EOL
1684,28753,None,2006-12-01,2,2006-12-26,2007-02-16,MS,30.0,2005-05-01,None,...,2006-01-01,2006-10-01,None,None,None,None,None,None,30.0,PCH


### No dataframes skate_usina e bdworgs são feitas as seguintes alterações:
 - Onde não não existe data na coluna DatMontagemOutorgado e existe na coluna DatConclusaoTorresOutorgado, o valor que está na coluna DatConclusaoTorresOutorgado passa para a coluna DatMontagemOutorgado.
 - Mesmo processo feito para as colunas DatConclusaoTorresOutorgado/DatMontagemOutorgado é feito para as colunas DatConclusaoTorresRealizado/DatMontagemRealizado

In [8]:
skate_usinas.loc[(skate_usinas.SigTipoGeracao == "EOL") & 
                 (skate_usinas.DatMontagemOutorgado.isnull()) &
                (skate_usinas.DatConclusaoTorresOutorgado.notnull()),"DatMontagemOutorgado"] =  skate_usinas.DatConclusaoTorresOutorgado


skate_usinas.loc[(skate_usinas.SigTipoGeracao == "EOL") & 
                 (skate_usinas.DatMontagemRealizado.isnull()) &
                (skate_usinas.DatConclusaoTorresRealizado.notnull()),"DatMontagemRealizado"] = skate_usinas.DatConclusaoTorresRealizado


bdworgs.loc[(bdworgs.SigTipoGeracao == "EOL") & 
                 (bdworgs.DatMontagemOutorgado.isnull()) &
                (bdworgs.DatConclusaoTorresOutorgado.notnull()),"DatMontagemOutorgado"] =  bdworgs.DatConclusaoTorresOutorgado


bdworgs.loc[(bdworgs.SigTipoGeracao == "EOL") & 
                 (bdworgs.DatMontagemRealizado.isnull()) &
                (bdworgs.DatConclusaoTorresRealizado.notnull()),"DatMontagemRealizado"] = bdworgs.DatConclusaoTorresRealizado


### No BD skate_ug, linhas com todas as colunas iguais tais como IdeUsinaOutorga, DatInicioOpTesteOutorgada... (exceto  MdaPotenciaUnitaria, que não necessariamente tem que ser igual em todas as linhas)  são agrupadas e são criadas duas outras colunas: 
- Quant_UG -> &rarr; Quantidade de UGs agrupadas
- Pot_UG -> &rarr; Soma da potência das UGs que foram agrupadas

#### Entrada: skate_ug
#### Saída: skate_ug_grouped

In [9]:
skate_ug_group_list = [x for x  in skate_ug.columns.to_list() if x not in ["MdaPotenciaUnitaria"]]
skate_ug_grouped = skate_ug.groupby(skate_ug_group_list,dropna=False).agg(
    Quant_UG = ("MdaPotenciaUnitaria","count"),
    Pot_UG = ("MdaPotenciaUnitaria","sum")
    
).reset_index()

### Realiza o processo acima para o bdworgs:
- Quant_UG -> &rarr; Quantidade de UGs agrupadas. Nesse caso, como o bdworgs, já possuía a coluna Quant_UG, os valores dessa coluna foram somados para os valores agrupados.
- Pot_UG -> &rarr; Soma da potência dos conjuntos de UGs que foram agrupados.

#### Entrada: bdworgs
#### Saída: bdworgs_ug_grouped

In [10]:
dummy_bdworgs = bdworgs.rename(columns={"Quant_UG":"Quant_UG_","Pot_UG":"Pot_UG_"})
dummy_bdworgs_group_list = [x for x  in dummy_bdworgs.columns.to_list() if x not in ["Quant_UG_","Pot_UG_"]]

bdworgs_ug_grouped = dummy_bdworgs.groupby(dummy_bdworgs_group_list,dropna=False).agg(
    Quant_UG = ("Quant_UG_","sum"),
    Pot_UG = ("Pot_UG_","sum")
    
).reset_index() 

### Checa se a quantidade de potência e de usinas nos dataframes iniciais e agrupados são iguais.

In [11]:
print(skate_ug_grouped.Quant_UG.sum() - skate_ug.MdaPotenciaUnitaria.count())
print(skate_ug_grouped.Pot_UG.sum() - skate_ug.MdaPotenciaUnitaria.sum())
print(bdworgs_ug_grouped.Pot_UG.sum() - bdworgs_ug_grouped.Pot_UG.sum())
print(bdworgs_ug_grouped.Quant_UG.count() - bdworgs_ug_grouped.Quant_UG.count())


0
0.0
0.0
0


### Junta bancos de dados skate_ug e skate_usina pela coluna IdeUsinaOutorga. Todos os atributos de cada usina são passados para cada UG pertecente a usina.

#### Entrada: skate_ug_grouped & skate_usinas
#### Saída: skate_merged

In [12]:
skate_merged  = pd.merge(skate_ug_grouped.drop(["SigTipoGeracao"],axis=1),
        skate_usinas,
        on="IdeUsinaOutorga",
         how="outer")
skate_merged.sample(10)

,IdeUsinaOutorga,DatInicioOpTesteOutorgada,DatUGInicioOpComerOutorgado,DatLiberacaoSFGTeste,DatLiberOpComerRealizado,Quant_UG,Pot_UG,DatOutorgaUsina,UFUsina,SigTipoGeracao,...,DatConcretagemOutorgado,DatConcretagemRealizado,DatMontagemOutorgado,DatMontagemRealizado,DatConclusaoTorresOutorgado,DatConclusaoTorresRealizado,DatEnchimentoOutorgado,DatEnchimentoRealizado,DatSisTransmissaoOutorgado,DatSisTransmissaoRealizado
971,34625,2021-07-16,2021-07-31,2021-07-07,2021-09-02,2,8400.00,2019-12-10,BA,EOL,...,2020-08-05,2020-10-22,2021-03-03,2021-01-26,None,2021-06-15,None,None,2019-08-31,2020-04-15
833,33975,2020-12-17,2021-01-17,2022-02-09,2022-03-18,1,15000.00,2015-04-22,PR,PCH,...,2019-08-10,2021-02-15,2020-01-22,2021-06-14,None,None,None,2021-11-16,2019-11-03,2021-09-14
1309,35722,2019-01-06,2019-01-14,NaN,NaN,1,440.00,2016-05-17,PA,UTE,...,None,None,2016-10-18,2016-12-05,None,None,None,None,2016-12-20,2016-12-05
3471,51636,2024-12-10,2025-01-10,NaN,NaN,12,41244.00,2021-08-24,CE,UFV,...,None,None,2024-05-10,None,None,None,None,None,2024-05-10,None
1592,37518,2021-05-24,2022-02-28,NaN,NaN,1,4600.00,2020-03-03,BA,UFV,...,None,None,2021-01-16,None,None,None,None,None,2020-09-02,None
1870,38127,2023-12-13,2024-05-15,NaN,NaN,8,49600.00,2022-04-12,PI,EOL,...,2023-03-06,None,2023-10-01,None,None,None,None,None,2023-02-06,None
1888,38143,2020-08-01,2020-10-01,2020-03-31,2020-05-30,9,37800.00,2019-03-12,RN,EOL,...,2019-09-17,2019-10-15,2020-05-01,2020-03-15,None,2020-03-31,None,None,2018-10-31,2018-10-31
2332,44299,2022-08-29,2022-12-29,NaN,NaN,20,32800.00,2020-04-28,PE,UFV,...,None,None,2021-12-17,None,None,None,None,None,2021-11-08,None
1379,36973,2020-08-01,2020-10-01,2020-06-06,2020-08-27,3,12600.00,2019-03-19,RN,EOL,...,2019-09-17,2019-12-04,2020-05-01,2020-06-15,None,2020-06-16,None,None,None,2018-10-31
3578,54631,2025-09-01,2026-01-01,NaN,NaN,33,49999.95,2022-04-19,MG,UFV,...,None,None,2024-11-01,None,None,None,None,None,2024-08-01,None


### Converte colunas cujos nomes iniciam em "Dat" para tipo de data. É necessário que as colunas estejam nomeadas no padrão do skate.

In [13]:
colunas_datas = []
for col in bdworgs_ug_grouped.columns:
    if col[0:3] == "Dat":
        colunas_datas.append(col)
        print(col)
        skate_merged[col] = pd.to_datetime(skate_merged[col],format="%Y-%m-%d",errors="coerce")
        bdworgs_ug_grouped[col] = pd.to_datetime(bdworgs_ug_grouped[col],format="%Y-%m-%d",errors="coerce")

DatInicioOpTesteOutorgada
DatUGInicioOpComerOutorgado
DatLiberacaoSFGTeste
DatLiberOpComerRealizado
DatInicioObraOutorgado
DatInicioObraRealizado
DatConcretagemOutorgado
DatConcretagemRealizado
DatMontagemOutorgado
DatMontagemRealizado
DatConclusaoTorresOutorgado
DatConclusaoTorresRealizado
DatEnchimentoOutorgado
DatEnchimentoRealizado
DatSisTransmissaoOutorgado
DatSisTransmissaoRealizado


### Checa quais colunas não estão presentes em cada um dos dataframes

In [14]:
print("Colunas que estão no skate, porém não no bdworgs:")
for col in skate_merged.columns:
    if col not in bdworgs.columns:
        print(col)
        
print("-"*10)
print("Colunas que estão no bdworgs, porém não no skate:")
for col in bdworgs.columns:
    if col not in skate_merged.columns:
        print(col)

Colunas que estão no skate, porém não no bdworgs:
DatOutorgaUsina
----------
Colunas que estão no bdworgs, porém não no skate:


### Junta BDs skate e bdworgs.
#### Entrada: skate_merged & bdworgs_ug_grouped
#### Saída: skate_merged

In [15]:
bdworgs_ug_grouped["origem"] = "bdworgs"
skate_merged["origem"] = "skate"
bd_combined = pd.concat([bdworgs_ug_grouped,skate_merged],axis=0,ignore_index=True)
#bd_combined = bdworgs_ug_grouped.copy(deep=True)
bd_combined["origem"] = bd_combined["origem"].astype("category")
bd_combined.sample(10)

,IdeUsinaOutorga,DatInicioOpTesteOutorgada,DatUGInicioOpComerOutorgado,DatLiberacaoSFGTeste,DatLiberOpComerRealizado,UFUsina,VlrPotenciaOutorgada,DatInicioObraOutorgado,DatInicioObraRealizado,DatConcretagemOutorgado,...,DatConclusaoTorresRealizado,DatEnchimentoOutorgado,DatEnchimentoRealizado,DatSisTransmissaoOutorgado,DatSisTransmissaoRealizado,SigTipoGeracao,Quant_UG,Pot_UG,origem,DatOutorgaUsina
5866,52324,2023-12-01,2024-01-01,NaT,NaT,GO,50000.0,2023-01-01,NaT,NaT,...,NaT,NaT,NaT,2023-02-01,NaT,UFV,16,50000.0,skate,2022-04-26
5766,51424,2023-03-01,2023-04-01,NaT,NaT,RR,2520.0,2022-06-20,NaT,NaT,...,NaT,NaT,NaT,2022-09-10,NaT,UTE,8,2520.0,skate,2021-09-08
4974,46737,2023-12-01,2024-01-31,NaT,NaT,BA,30000.0,2022-08-01,NaT,NaT,...,NaT,NaT,NaT,2023-02-01,NaT,UFV,12,30000.0,skate,2021-09-21
4092,37820,2023-06-20,2023-06-28,NaT,NaT,SC,9600.0,2022-02-12,2021-11-13,2022-07-06,...,NaT,2022-01-01,NaT,2023-02-01,NaT,PCH,1,3200.0,skate,2019-01-17
5306,48559,2023-10-27,2023-11-27,NaT,NaT,MG,41244.0,2022-09-27,NaT,NaT,...,NaT,NaT,NaT,2022-12-27,NaT,UFV,12,41244.0,skate,2021-06-15
2947,33015,2022-11-17,2023-01-01,2020-08-01,2020-12-31,PI,44100.0,2022-03-15,2019-05-15,2022-05-01,...,2020-11-07,NaT,NaT,2022-04-01,2019-05-06,EOL,3,9450.0,skate,2018-05-22
5693,50082,2024-01-06,2024-01-16,NaT,NaT,RN,42000.0,2023-08-17,NaT,2023-09-27,...,NaT,NaT,NaT,NaT,NaT,EOL,2,12000.0,skate,2021-07-06
673,29709,NaT,NaT,2014-08-08,2014-08-19,MG,54.0,NaT,2013-09-30,NaT,...,NaT,NaT,NaT,NaT,2013-09-30,UTE,1,40.0,bdworgs,NaN
3697,35861,2021-12-01,2021-12-30,2021-08-17,2021-10-30,CE,30933.0,2021-01-01,2020-08-10,NaT,...,NaT,NaT,NaT,2021-09-01,2021-01-15,UFV,9,30933.0,skate,2018-08-20
1319,30902,2015-11-30,2015-12-31,2015-11-06,2015-12-09,RN,24.0,2015-05-01,2014-12-01,2015-07-01,...,2015-04-24,NaT,NaT,2015-03-01,2015-02-15,EOL,3,9.0,bdworgs,NaN


In [16]:
bd_combined.to_parquet("bd_combined.gzip")